In [70]:
import pandas as pd
import pickle

# Load EP Model
with open('ep_model.pkl', 'rb') as f:
    ep_model = pickle.load(f)

# Load Defender Data with GIZ% Predictions
df = pd.read_csv('defender_giz_predictions.csv')
print(f"      ✓ Loaded {len(df):,} rows")
print(f"      → {df['nfl_id'].nunique():,} unique players")
print(f"      → {df.groupby(['game_id','play_id']).ngroups:,} unique plays")
print(f"\n      Columns available:")
for col in df.columns:
    print(f"        - {col}")

      ✓ Loaded 22,185 rows
      → 687 unique players
      → 9,072 unique plays

      Columns available:
        - game_id
        - play_id
        - nfl_id
        - frame_id
        - player_name
        - player_role
        - player_side
        - x_flow
        - y_rotated
        - start_s
        - start_a
        - start_dir
        - start_o
        - num_frames_output
        - team_coverage_man_zone
        - ball_in_air_time
        - angle_from_deep_rad
        - angle_from_deep
        - dist_to_ball
        - rotation_angle
        - flow_factor
        - dir_scaled
        - o_scaled
        - accel_towards_ball
        - velocity_towards_ball
        - target_made_zone
        - week
        - pred_prob
        - giz_oe


In [72]:
import numpy as np

# Load supplementary data for play outcomes
try:
    supp_df = pd.read_csv('supplementary_data.csv', low_memory=False)
    
    # Filter to zone coverage only
    if 'teamCoverageManZone' in supp_df.columns:
        supp_df = supp_df[supp_df['teamCoverageManZone'] == 'ZONE_COVERAGE'].copy()
    
    # Keep only play-level info we need
    outcome_cols = ['game_id', 'play_id', 'pass_result', 'playResult', 
                    'down', 'yardsToGo', 'absoluteYardlineNumber', 
                    'gameClock', 'quarter', 'preSnapHomeScore', 
                    'preSnapVisitorScore', 'possessionTeam']
    
    # Only keep columns that exist
    outcome_cols = [c for c in outcome_cols if c in supp_df.columns]
    
    play_outcomes = supp_df[outcome_cols].drop_duplicates(subset=['game_id', 'play_id'])
    
    print(f"      ✓ Loaded {len(play_outcomes):,} zone coverage plays")
    
except FileNotFoundError:
    print("      ✗ supplementary_data.csv not found")
    play_outcomes = None

# Merge outcomes with defender data
if df is not None and play_outcomes is not None:
    df = df.merge(play_outcomes, on=['game_id', 'play_id'], how='left')
    print(f"      → pass_result column: {df['pass_result'].notna().sum():,}/{len(df):,} rows have outcome")
    
    # Outcome distribution
    if 'pass_result' in df.columns:
        print(f"\n      Outcome Distribution:")
        play_level_outcomes = df.drop_duplicates(subset=['game_id', 'play_id'])
        outcome_counts = play_level_outcomes['pass_result'].value_counts()
        total_plays_with_defenders = len(play_level_outcomes)  # Use THIS as denominator
        
        for outcome, count in outcome_counts.items():
            pct = 100 * count / total_plays_with_defenders  # Fixed!
            print(f"        {outcome}: {count:>5,} plays ({pct:>5.1f}%)")
        
        print(f"      Total: {outcome_counts.sum():,} plays with evaluated defenders")


# Count defenders in zone per play
if df is not None and 'target_made_zone' in df.columns:
    # For each play, count how many defenders actually reached the zone
    defenders_per_play = df.groupby(['game_id', 'play_id'])['target_made_zone'].sum().reset_index()
    defenders_per_play.columns = ['game_id', 'play_id', 'num_defenders_in_zone']
    
    # Merge back to main dataframe
    df = df.merge(defenders_per_play, on=['game_id', 'play_id'], how='left')
    
    # Show distribution
    play_level = df.drop_duplicates(subset=['game_id', 'play_id'])
    zone_dist = play_level['num_defenders_in_zone'].value_counts().sort_index()
    
    print(f"\n      Distribution of Defenders in Zone:")
    print(f"      {'Defenders':<12} {'Plays':<10} {'Percent':<10}")
    print(f"      {'-'*35}")
    
    total_plays = len(play_level)
    for num_def, count in zone_dist.items():
        pct = 100 * count / total_plays
        print(f"      {int(num_def):<12} {count:>6,}     {pct:>5.1f}%")

# Calculate outcome probabilities
if df is not None and 'pass_result' in df.columns:
    play_level = df.drop_duplicates(subset=['game_id', 'play_id']).copy()
    
    # Create defender category
    play_level['def_category'] = play_level['num_defenders_in_zone'].apply(
        lambda x: '2+' if x >= 2 else str(int(x))
    )
    categories = ['0', '1', '2+']
    
    # Calculate outcome probabilities
    outcome_probs = play_level.groupby(['def_category', 'pass_result']).size().unstack(fill_value=0)
    outcome_probs_pct = outcome_probs.div(outcome_probs.sum(axis=1), axis=0)
    
    # Print outcome breakdown
    outcome_probs_pct = outcome_probs_pct.reindex(categories)
    print(f"\n{outcome_probs_pct.round(3).to_string()}")
    outcome_probs_reordered = outcome_probs.reindex(categories)
    print(f"\n{outcome_probs_reordered.to_string()}")
    
    # Convert to dictionary
    outcome_prob_dict = {}
    for def_cat in outcome_probs_pct.index:
        outcome_prob_dict[def_cat] = outcome_probs_pct.loc[def_cat].to_dict()

      ✓ Loaded 18,009 zone coverage plays
      → pass_result column: 22,185/22,185 rows have outcome

      Outcome Distribution:
        C: 6,379 plays ( 70.3%)
        I: 2,435 plays ( 26.8%)
        IN:   258 plays (  2.8%)
      Total: 9,072 plays with evaluated defenders

      Distribution of Defenders in Zone:
      Defenders    Plays      Percent   
      -----------------------------------
      0             7,394      81.5%
      1             1,581      17.4%
      2                96       1.1%
      3                 1       0.0%

pass_result       C      I     IN
def_category                     
0             0.771  0.220  0.009
1             0.414  0.477  0.109
2+            0.268  0.536  0.196

pass_result      C     I   IN
def_category                 
0             5699  1629   66
1              654   754  173
2+              26    52   19


In [74]:
import re

# Parse INT return yards
def extract_int_return_yards(play_desc):
    if pd.isna(play_desc):
        return np.nan
    
    # Pattern 1: "for X yards"
    match = re.search(r'for (\d+) yards?', play_desc, re.IGNORECASE)
    if match:
        return int(match.group(1))
    
    # Pattern 2: "for no gain"
    if re.search(r'for no gain', play_desc, re.IGNORECASE):
        return 0
    
    return np.nan

# Load supplementary data and calculate YAC/INT return yds
try:
    supp_df = pd.read_csv('supplementary_data.csv', low_memory=False)
    
    # Filter to zone coverage
    if 'teamCoverageManZone' in supp_df.columns:
        supp_df = supp_df[supp_df['teamCoverageManZone'] == 'ZONE_COVERAGE'].copy()
    
    # Keep only play-level data
    play_cols = ['game_id', 'play_id', 'pass_result', 'pass_length', 
                 'pre_penalty_yards_gained', 'play_description']
    play_cols = [c for c in play_cols if c in supp_df.columns]
    
    supp_plays = supp_df[play_cols].drop_duplicates(subset=['game_id', 'play_id'])
    
    # Calculate YAC for catches
    completions_mask = supp_plays['pass_result'] == 'C'
    
    supp_plays.loc[completions_mask, 'yac'] = (
            supp_plays.loc[completions_mask, 'pre_penalty_yards_gained'] - 
            supp_plays.loc[completions_mask, 'pass_length']
        )
    print(f"      ✓ YAC calculated for {completions_mask.sum():,} completions")
        
    # Calculate return yards for INTS
    int_mask = supp_plays['pass_result'] == 'IN'
    
    supp_plays.loc[int_mask, 'int_return_yards'] = (
        supp_plays.loc[int_mask, 'play_description'].apply(extract_int_return_yards)
    )
        
    parsed = supp_plays.loc[int_mask, 'int_return_yards'].notna().sum()
    total_ints = int_mask.sum()
        
    print(f"      ✓ Parsed return yards for {parsed:,}/{total_ints:,} interceptions ({100*parsed/total_ints:.1f}%)")
        
    # Show sample
    if parsed > 0:
        print(f"\n      Sample Parsed Interceptions:")
        sample = supp_plays[int_mask & supp_plays['int_return_yards'].notna()].head(3)
        for idx, row in sample.iterrows():
            desc = row['play_description'][:70] + "..." if len(row['play_description']) > 70 else row['play_description']
            yards = row['int_return_yards']
            print(f"        {yards:>3.0f} yds: {desc}")
    else:
        print(f"      ✗ No play_description column for interception parsing")
    
except FileNotFoundError:
    print("      ✗ supplementary_data.csv not found")
    supp_plays = None

# Merge with defender counts
if supp_plays is not None:
    # Get play-level data with defender counts from main df
    play_level = df.drop_duplicates(subset=['game_id', 'play_id']).copy()
    
    # Merge YAC and return data
    play_level = play_level.merge(
        supp_plays[['game_id', 'play_id', 'yac', 'int_return_yards']], 
        on=['game_id', 'play_id'], 
        how='left'
    )
    
    # Add defender category
    play_level['def_category'] = play_level['num_defenders_in_zone'].apply(
        lambda x: '2+' if x >= 2 else str(int(x))
    )
    categories = ['0', '1', '2+']
# Calculate averages based on # of defenders in zone

# YAC on catches
completions = play_level[play_level['pass_result'] == 'C'].copy()
print(f"\n      Completions: {len(completions):,}")
if 'yac' in completions.columns:
    yac_by_def = completions.groupby('def_category')['yac'].agg(['mean', 'count']).round(2)
    
    print(f"\n      Average YAC on Completions:")
    print(f"      {'-'*50}")
    print(f"      {'Defenders':<12} {'Avg YAC':<10} {'N Plays':<10}")
    print(f"      {'-'*50}")
    
    for def_cat in categories:
        if def_cat in yac_by_def.index:
            avg_yac = yac_by_def.loc[def_cat, 'mean']
            n_plays = int(yac_by_def.loc[def_cat, 'count'])
            print(f"      {def_cat:<12} {avg_yac:>6.2f} yds   {n_plays:>6,}")
        else:
            print(f"      {def_cat:<12} {'N/A':<10} {'0':>6}")
    
    # Save as dictionary
    yac_dict = yac_by_def['mean'].to_dict()
else:
    yac_dict = {}
    print(f"      ✗ YAC column not available")

# Return Yards on Interceptions
interceptions = play_level[play_level['pass_result'] == 'IN'].copy()
print(f"\n      Interceptions: {len(interceptions):,}")

if 'int_return_yards' in interceptions.columns and len(interceptions) > 0:
    
    # Average/median
    avg_int_return_yards = interceptions['int_return_yards'].mean()
    median_int_return = interceptions['int_return_yards'].median()
    n_parsed = interceptions['int_return_yards'].notna().sum()
    
    print(f"\n      Interception Return Statistics (ALL):")
    print(f"      {'-'*50}")
    print(f"      Average Return:      {avg_int_return_yards:>6.2f} yards")
    print(f"      Median Return:       {median_int_return:>6.1f} yards")
    print(f"      Successfully Parsed: {n_parsed:>6,}/{len(interceptions):,} ({100*n_parsed/len(interceptions):.1f}%)")
    
    # For expected points, use the single average for all scenarios
    return_dict = {cat: avg_int_return_yards for cat in categories}
    
else:
    avg_int_return_yards = 0.0
    return_dict = {}
    print(f"      ✗ No interception return data available")

# Summary
print(f"\nYAC on Completions:")
for def_cat in categories:
    yac_val = yac_dict.get(def_cat, None)
    if yac_val is not None:
        print(f"  {def_cat} defenders: {yac_val:.2f} yards")
    else:
        print(f"  {def_cat} defenders: N/A")

print(f"\nReturn Yards on Interceptions:")
print(f"  All scenarios: {avg_int_return_yards:.2f} yards")

      ✓ YAC calculated for 12,470 completions
      ✓ Parsed return yards for 407/433 interceptions (94.0%)

      Sample Parsed Interceptions:
         50 yds: (11:04) (Shotgun) P.Mahomes pass short right intended for K.Toney INTE...
          3 yds: (1:10) (Shotgun) B.Young pass short middle intended for H.Hurst INTERC...
          0 yds: (6:48) (Shotgun) B.Young pass short middle intended for T.Marshall INT...

      Completions: 6,379

      Average YAC on Completions:
      --------------------------------------------------
      Defenders    Avg YAC    N Plays   
      --------------------------------------------------
      0              3.91 yds    5,699
      1              3.53 yds      654
      2+             0.81 yds       26

      Interceptions: 258

      Interception Return Statistics (ALL):
      --------------------------------------------------
      Average Return:       14.87 yards
      Median Return:          8.0 yards
      Successfully Parsed:    246/258 (95.

In [76]:
# Load the EP features
ep_features = pd.read_csv('ep_model_features.csv')

print(f"Loaded EP features: {len(ep_features):,} plays")
print(f"Columns: {list(ep_features.columns)}")

# Merge with your defender dataframe
df = df.merge(
    ep_features,
    on=['game_id', 'play_id'],
    how='left'
)

# Handle duplicate 'down' column from merge
if 'down_x' in df.columns and 'down_y' in df.columns:
    df['down'] = df['down_x']  # Keep one version
    df = df.drop(columns=['down_x', 'down_y'])
    print("✓ Fixed duplicate 'down' columns")
elif 'down_x' in df.columns:
    df['down'] = df['down_x']
    df = df.drop(columns=['down_x'])
elif 'down_y' in df.columns:
    df['down'] = df['down_y']
    df = df.drop(columns=['down_y'])

print(f"\ndf now has {len(df.columns)} columns")
print(f" Missing EP features for {df['yardline'].isna().sum():,} rows")

Loaded EP features: 18,009 plays
Columns: ['game_id', 'play_id', 'down', 'yards_to_go', 'yardline', 'score_diff', 'time_remaining_game', 'time_remaining_half', 'end_of_half', 'hurry_up_time', 'time_score_interaction', 'quarter_cat']
✓ Fixed duplicate 'down' columns

df now has 42 columns
 Missing EP features for 0 rows


In [78]:
# Function to create EP model features
def create_ep_features(down, yards_to_go, yardline, score_diff, 
                       time_remaining_game, time_remaining_half, 
                       quarter_cat, end_of_half, hurry_up_time):
    
    time_score_interaction = time_remaining_game * score_diff
    
    features = pd.DataFrame([{
        'down': down,
        'yards_to_go': yards_to_go,
        'yardline': yardline,
        'score_diff': score_diff,
        'time_remaining_game': time_remaining_game,
        'time_remaining_half': time_remaining_half,
        'end_of_half': end_of_half,
        'hurry_up_time': hurry_up_time,
        'time_score_interaction': time_score_interaction,
        'quarter_cat': quarter_cat
    }])
    
    # Check data types
    for col in features.columns:
        features[col] = features[col].astype('int64')
    
    return features

# Function to calculate the EP of each outcome on each play
def calculate_outcome_eps(row, ep_model, yac_dict, avg_int_return):
    
    def_cat = row['def_category']
    current_down = row['down']
    los_yardline = row['yardline']
    pass_length = row['pass_length'] if pd.notna(row['pass_length']) else 0
    yards_to_go = row['yards_to_go']
    yardline_to_gain = los_yardline - yards_to_go  # First down marker
    
    # Function to update clock after play
    def update_time(time_remaining_game, time_remaining_half, play_duration, 
                    hurry_up, end_of_half, quarter_cat, is_completion=False):
        old_time_half = time_remaining_half
        new_time_half = time_remaining_half - play_duration
        new_time_game = time_remaining_game - play_duration
        new_quarter = quarter_cat
        new_end_of_half = end_of_half
        new_hurry_up = hurry_up
        
        # 2-minute warning (completions only)
        if is_completion and old_time_half > 120 and new_time_half < 120:
            new_time_half = 120
            new_time_game = time_remaining_game - (old_time_half - 120)
        
        # Quarter transitions (1st->2nd or 3rd->4th)
        if old_time_half > 900 and new_time_half <= 900:
            new_quarter = quarter_cat + 1
            new_time_half = 900
            if new_quarter == 2:
                new_time_game = 2700
            elif new_quarter == 4:
                new_time_game = 900
        
        # Check if half ended
        if new_time_half <= 0:
            return new_time_game, new_time_half, new_quarter, new_end_of_half, new_hurry_up, True
        
        # Update special time categories (end_of_half, hurry_up_time)
        if new_time_half <= 120:
            new_end_of_half = 1
        if new_time_game <= 300:
            new_hurry_up = 1
        
        return new_time_game, new_time_half, new_quarter, new_end_of_half, new_hurry_up, False
    
    # Catches
    catch_location = los_yardline - pass_length
    
    # Determine completion time runoff
    if row['hurry_up_time'] or row['end_of_half']:
        completion_duration = 15
    else:
        completion_duration = 30
    
    if catch_location <= 0:
        ep_catch = -7.0
    else:
        expected_yac = yac_dict.get(def_cat, 0)
        final_position = catch_location - expected_yac
        
        if final_position <= 0:
            catch_yardline = catch_location / 2
        else:
            catch_yardline = final_position
        
        catch_yardline = max(catch_yardline, 1)
        
        # Check if gains first down
        if catch_yardline <= yardline_to_gain:
            # First down
            next_down = 1
            next_ytg = 10
            
            new_time_game, new_time_half, new_quarter, new_end_of_half, new_hurry_up, half_ended = update_time(
                row['time_remaining_game'], 
                row['time_remaining_half'],
                completion_duration,
                row['hurry_up_time'],
                row['end_of_half'],
                row['quarter_cat'],
                is_completion=True
            )
            
            if half_ended:
                ep_catch = 0
            else:
                catch_features = create_ep_features(
                    down=next_down, 
                    yards_to_go=next_ytg,
                    yardline=catch_yardline,
                    score_diff=row['score_diff'],
                    time_remaining_game=new_time_game,
                    time_remaining_half=new_time_half,
                    quarter_cat=new_quarter,
                    end_of_half=new_end_of_half,
                    hurry_up_time=new_hurry_up
                )
                ep_catch = -ep_model.predict(catch_features.values)[0]
        
        elif current_down == 4:
            # Turnover on downs
            catch_yardline_defense = 100 - catch_yardline
            
            new_time_game, new_time_half, new_quarter, new_end_of_half, new_hurry_up, half_ended = update_time(
                row['time_remaining_game'], 
                row['time_remaining_half'],
                completion_duration,
                row['hurry_up_time'],
                row['end_of_half'],
                row['quarter_cat'],
                is_completion=True
            )
            
            if half_ended:
                ep_catch = 0
            else:
                catch_features = create_ep_features(
                    down=1,
                    yards_to_go=10,
                    yardline=catch_yardline_defense,
                    score_diff=-row['score_diff'],
                    time_remaining_game=new_time_game,
                    time_remaining_half=new_time_half,
                    quarter_cat=new_quarter,
                    end_of_half=new_end_of_half,
                    hurry_up_time=new_hurry_up
                )
                ep_catch = ep_model.predict(catch_features.values)[0]
        
        else:
            # Didn't convert, not 4th down
            next_down = current_down + 1
            next_ytg = catch_yardline - yardline_to_gain
            
            new_time_game, new_time_half, new_quarter, new_end_of_half, new_hurry_up, half_ended = update_time(
                row['time_remaining_game'], 
                row['time_remaining_half'],
                completion_duration,
                row['hurry_up_time'],
                row['end_of_half'],
                row['quarter_cat'],
                is_completion=True
            )
            
            if half_ended:
                ep_catch = 0
            else:
                catch_features = create_ep_features(
                    down=next_down, 
                    yards_to_go=next_ytg,
                    yardline=catch_yardline,
                    score_diff=row['score_diff'],
                    time_remaining_game=new_time_game,
                    time_remaining_half=new_time_half,
                    quarter_cat=new_quarter,
                    end_of_half=new_end_of_half,
                    hurry_up_time=new_hurry_up
                )
                ep_catch = -ep_model.predict(catch_features.values)[0]
    
    # Incompletions
    incompletion_duration = 4
    incomp_ball_location = los_yardline - pass_length
    
    new_time_game, new_time_half, new_quarter, new_end_of_half, new_hurry_up, half_ended = update_time(
        row['time_remaining_game'], 
        row['time_remaining_half'],
        incompletion_duration,
        row['hurry_up_time'],
        row['end_of_half'],
        row['quarter_cat'],
        is_completion=False
    )
    
    if half_ended:
        ep_incomp = 0
    elif current_down == 4:
        incomp_yardline = 100 - los_yardline
        incomp_features = create_ep_features(
            down=1,
            yards_to_go=10,
            yardline=incomp_yardline,
            score_diff=-row['score_diff'],
            time_remaining_game=new_time_game,
            time_remaining_half=new_time_half,
            quarter_cat=new_quarter,
            end_of_half=new_end_of_half,
            hurry_up_time=new_hurry_up
        )
        ep_incomp = ep_model.predict(incomp_features.values)[0]
    else:
        incomp_features = create_ep_features(
            down=current_down + 1,
            yards_to_go=yards_to_go,
            yardline=los_yardline,
            score_diff=row['score_diff'],
            time_remaining_game=new_time_game,
            time_remaining_half=new_time_half,
            quarter_cat=new_quarter,
            end_of_half=new_end_of_half,
            hurry_up_time=new_hurry_up
        )
        ep_incomp = -ep_model.predict(incomp_features.values)[0]
    
    # Interceptions
    interception_duration = 6
    int_catch_location = los_yardline - pass_length
    
    new_time_game, new_time_half, new_quarter, new_end_of_half, new_hurry_up, half_ended = update_time(
        row['time_remaining_game'], 
        row['time_remaining_half'],
        interception_duration,
        row['hurry_up_time'],
        row['end_of_half'],
        row['quarter_cat'],
        is_completion=False
    )
    
    if half_ended:
        ep_int = 0
    elif int_catch_location <= 0:
        int_features = create_ep_features(
            down=1,
            yards_to_go=10,
            yardline=80,
            score_diff=-row['score_diff'],
            time_remaining_game=new_time_game,
            time_remaining_half=new_time_half,
            quarter_cat=new_quarter,
            end_of_half=new_end_of_half,
            hurry_up_time=new_hurry_up
        )
        ep_int = ep_model.predict(int_features.values)[0]
    else:
        defense_yardline = 100 - int_catch_location
        final_defense_position = defense_yardline - avg_int_return
        
        if final_defense_position <= 0:
            int_yardline_defense = defense_yardline / 2
        else:
            int_yardline_defense = final_defense_position
        
        int_yardline_defense = max(int_yardline_defense, 1)
        int_features = create_ep_features(
            down=1,
            yards_to_go=10,
            yardline=int_yardline_defense,
            score_diff=-row['score_diff'],
            time_remaining_game=new_time_game,
            time_remaining_half=new_time_half,
            quarter_cat=new_quarter,
            end_of_half=new_end_of_half,
            hurry_up_time=new_hurry_up
        )
        ep_int = ep_model.predict(int_features.values)[0]
    
    return ep_catch, ep_incomp, ep_int

# Function to calculate expected EP for each play
def calculate_expected_ep(row, ep_catch, ep_incomp, ep_int, outcome_prob_dict):
    
    actual_defenders = row['num_defenders_in_zone']
    defender_reached = row['target_made_zone']
    
    if not defender_reached:
        if actual_defenders == 0:
            no_reach_def_cat = '0'
            reach_def_cat = '1'
            reach_split = 1  # Would be only defender
        else:
            no_reach_def_cat = str(min(actual_defenders, 1))
            reach_def_cat = '2+' if actual_defenders >= 1 else '1'
            reach_split = 2  # Would be 2nd defender (split credit)
    else:
        if actual_defenders == 1:
            no_reach_def_cat = '0'
            reach_def_cat = '1'
            reach_split = 1  # Only defender
        else:
            no_reach_def_cat = '1'
            reach_def_cat = '2+'
            reach_split = actual_defenders  # One of 2 defenders
    
    # Calculate expected EP for "didn't reach" scenario
    probs_no_reach = outcome_prob_dict[no_reach_def_cat]
    ep_no_reach = (
        probs_no_reach['C'] * ep_catch +
        probs_no_reach['I'] * ep_incomp +
        probs_no_reach['IN'] * ep_int
    )
    
    # Calculate expected EP for "reached" scenario
    probs_reach = outcome_prob_dict[reach_def_cat]
    ep_reach = (
        probs_reach['C'] * ep_catch +
        probs_reach['I'] * ep_incomp +
        probs_reach['IN'] * ep_int
    )
    
    # Split credit/blame in reach scenario if would be one of multiple defenders
    ep_reach = ep_reach / reach_split
    
    # Overall expected EP with GIZ% weighting
    reach_prob = row['pred_prob']
    expected_ep = (1 - reach_prob) * ep_no_reach + reach_prob * ep_reach
    
    return expected_ep

# Function to calculate actual EP created on each play
def calculate_actual_ep(row, ep_catch, ep_incomp, ep_int, outcome_prob_dict):
    
    actual_defenders = row['num_defenders_in_zone']
    defender_reached = row['target_made_zone']
    actual_outcome = row['pass_result']
    
    if defender_reached:
        # Defender was in the zone - get credit/blame for actual outcome
        if actual_outcome == 'C':
            actual_ep = ep_catch
        elif actual_outcome == 'I':
            actual_ep = ep_incomp
        else:
            actual_ep = ep_int
        
        # Split the credit/blame when >1 defenders reached zone
        if actual_defenders > 1:
            actual_ep = actual_ep / actual_defenders
            
    else:
        # Defender wasn't in zone - use probabilities based on who else reached
        if actual_defenders == 0:
            def_cat = '0'
        elif actual_defenders == 1:
            def_cat = '1'
        else:
            def_cat = '2+'
        
        probs = outcome_prob_dict[def_cat]
        actual_ep = (
            probs['C'] * ep_catch +
            probs['I'] * ep_incomp +
            probs['IN'] * ep_int
        )
    
    return actual_ep


# Filter plays
print("Filtering plays for metric calculation...")
df_metric = df[
    ((df['target_made_zone'] == 0) & (df['num_defenders_in_zone'] <= 1)) |
    ((df['target_made_zone'] == 1) & (df['num_defenders_in_zone'] <= 2))
].copy()

# Create column to split plays into categories based on # of defenders
df_metric['def_category'] = df_metric['num_defenders_in_zone'].apply(
    lambda x: '2+' if x >= 2 else str(int(x))
)

# Load pass_length from supplementary data
print("Loading pass_length data...")
supp_df = pd.read_csv('supplementary_data.csv', low_memory=False)

# Filter to zone coverage and get pass_length
if 'teamCoverageManZone' in supp_df.columns:
    supp_df = supp_df[supp_df['teamCoverageManZone'] == 'ZONE_COVERAGE'].copy()

pass_length_data = supp_df[['game_id', 'play_id', 'pass_length']].drop_duplicates(subset=['game_id', 'play_id'])

# Merge with df_metric
df_metric = df_metric.merge(pass_length_data, on=['game_id', 'play_id'], how='left')

print(f"✓ Loaded pass_length for {df_metric['pass_length'].notna().sum():,}/{len(df_metric):,} rows")

print(f"✓ {len(df_metric):,} defender-play observations included")
print(f"✓ {df_metric.groupby(['game_id','play_id']).ngroups:,} unique plays")

# Calculate metric
print("\nCalculating DRPVOE metric...")
results = []

from tqdm import tqdm  # Progress bar

total_rows = len(df_metric)
for idx, (_, row) in tqdm(enumerate(df_metric.iterrows()), total=total_rows, desc="Processing plays"):
    ep_c, ep_i, ep_in = calculate_outcome_eps(row, ep_model, yac_dict, avg_int_return_yards)
    expected_ep = calculate_expected_ep(row, ep_c, ep_i, ep_in, outcome_prob_dict)
    actual_ep = calculate_actual_ep(row, ep_c, ep_i, ep_in, outcome_prob_dict)
    
    drpvoe = actual_ep - expected_ep
    
    results.append({
        'nfl_id': row['nfl_id'],
        'player_name': row['player_name'],
        'game_id': row['game_id'],
        'play_id': row['play_id'],
        'actual_ep': actual_ep,
        'expected_ep': expected_ep,
        'drpvoe': drpvoe
    })
    
    # Print every 1000 rows
    if (idx + 1) % 1000 == 0:
        print(f"Processed {idx + 1:,}/{total_rows:,} rows...")

metric_df = pd.DataFrame(results)

# Aggregate by player
player_totals = metric_df.groupby(['nfl_id', 'player_name']).agg({
    'drpvoe': 'sum',
    'play_id': 'count'
}).rename(columns={'play_id': 'n_plays'}).sort_values('drpvoe', ascending=False)

print("\nTop 10 Players by DRPVOE:")
print(player_totals.head(10))

Filtering plays for metric calculation...
Loading pass_length data...
✓ Loaded pass_length for 22,002/22,002 rows
✓ 22,002 defender-play observations included
✓ 9,071 unique plays

Calculating DRPVOE metric...


Processing plays:   5%|▉                  | 1017/22002 [00:07<02:26, 143.52it/s]

Processed 1,000/22,002 rows...


Processing plays:   9%|█▋                 | 2020/22002 [00:14<02:25, 137.29it/s]

Processed 2,000/22,002 rows...


...

Processed 20,000/22,002 rows...


Processing plays:  96%|█████████████████▏| 21018/22002 [02:28<00:06, 141.21it/s]

Processed 21,000/22,002 rows...


Processing plays: 100%|██████████████████| 22002/22002 [02:35<00:00, 141.88it/s]


Processed 22,000/22,002 rows...

Top 10 Players by DRPVOE:
                            drpvoe  n_plays
nfl_id player_name                         
47872  Bobby Okereke     19.809848       71
44911  Rasul Douglas     17.239644       91
52493  Julian Blackmon   16.757537      112
52458  Jaylon Johnson    15.902533       63
46757  Charvarius Ward   14.773454       98
54525  Cam Taylor-Britt  14.546586       66
53505  Paulson Adebo     14.168950       89
54562  Kerby Joseph      14.158110       76
52627  Geno Stone        13.658871       82
52444  Xavier McKinney   13.384616       61


In [80]:
# Audit of several plays to verify metric creation process
def audit_play_calculation_detailed(row, ep_model, yac_dict, avg_int_return, outcome_prob_dict):
    
    print("="*100)
    print(f"PLAY AUDIT: {row['player_name']} - Game {row['game_id']}, Play {row['play_id']}")
    print("="*100)
    
    # Initial state
    print(f"\nINITIAL PLAY STATE:")
    print(f"  Down: {row['down']}")
    print(f"  Yards to Go: {row['yards_to_go']}")
    print(f"  Yardline (yards to end zone): {row['yardline']}")
    print(f"  Yardline to Gain (first down marker): {row['yardline'] - row['yards_to_go']}")
    print(f"  Pass Length (air yards): {row['pass_length']}")
    print(f"  Score Diff: {row['score_diff']}")
    print(f"  Time Remaining Game: {row['time_remaining_game']} seconds")
    print(f"  Time Remaining Half: {row['time_remaining_half']} seconds")
    print(f"  Quarter: {row['quarter_cat']}")
    print(f"  End of Half Flag: {row['end_of_half']}")
    print(f"  Hurry Up Time Flag: {row['hurry_up_time']}")
    print(f"  Defender Category (# def in zone): {row['def_category']}")
    print(f"  Defender Reached Zone: {bool(row['target_made_zone'])}")
    print(f"  Total Defenders in Zone: {int(row['num_defenders_in_zone'])}")
    print(f"  Actual Outcome: {row['pass_result']}")
    print(f"  Probability of Reaching Zone: {row['pred_prob']:.3f}")
    
    # Calculate EPs for each outcome
    print(f"\nEXPECTED POINTS BY OUTCOME:")
    ep_c, ep_i, ep_in = calculate_outcome_eps(row, ep_model, yac_dict, avg_int_return)
    print(f"  Completion (C):   {ep_c:>7.3f} EP")
    print(f"  Incompletion (I): {ep_i:>7.3f} EP")
    print(f"  Interception (IN):{ep_in:>7.3f} EP")
    
    # Show catch calculation details
    print(f"\nCATCH OUTCOME DETAILS:")
    catch_location = row['yardline'] - row['pass_length']
    print(f"  Catch Location: {row['yardline']} - {row['pass_length']} = {catch_location} yards to end zone")
    
    if catch_location <= 0:
        print(f"  Ball caught in end zone = TOUCHDOWN (-7.0 EP)")
    else:
        expected_yac = yac_dict.get(row['def_category'], 0)
        print(f"  Expected YAC (for {row['def_category']} defenders): {expected_yac} yards")
        final_position = catch_location - expected_yac
        print(f"  Final Position after YAC: {catch_location} - {expected_yac} = {final_position}")
        
        if final_position <= 0:
            catch_yardline = catch_location / 2
            print(f"  Would score TD with YAC, use halfway: {catch_yardline:.1f} yards")
        else:
            catch_yardline = final_position
            print(f"  Final catch position: {catch_yardline:.1f} yards to end zone")
        
        yardline_to_gain = row['yardline'] - row['yards_to_go']
        print(f"  First Down Marker: {yardline_to_gain} yards to end zone")
        
        if catch_yardline <= yardline_to_gain:
            print(f"  FIRST DOWN ACHIEVED (catch at {catch_yardline:.1f} ≤ {yardline_to_gain})")
            print(f"   Next play: 1st & 10 at {catch_yardline:.1f}")
        elif row['down'] == 4:
            print(f"  4TH DOWN FAILED CONVERSION (catch at {catch_yardline:.1f} > {yardline_to_gain})")
            print(f"   TURNOVER ON DOWNS")
            print(f"   Defense gets ball at {100 - catch_yardline:.1f} yards (their perspective)")
        else:
            next_down = row['down'] + 1
            next_ytg = catch_yardline - yardline_to_gain
            print(f"  No first down, advance down")
            print(f"   Next play: {next_down} & {next_ytg:.1f} at {catch_yardline:.1f}")
    
    # Show incompletion details
    print(f"\nINCOMPLETION OUTCOME DETAILS:")
    if row['down'] == 4:
        print(f"  4TH DOWN INCOMPLETION = TURNOVER ON DOWNS")
        print(f"   Defense gets ball at {100 - row['yardline']} yards (their perspective)")
    else:
        print(f"  Advance to {row['down'] + 1} & {row['yards_to_go']} at {row['yardline']}")
    
    # Show interception details
    print(f"\nINTERCEPTION OUTCOME DETAILS:")
    int_catch_location = row['yardline'] - row['pass_length']
    if int_catch_location <= 0:
        print(f"  INT in end zone = TOUCHBACK")
        print(f"  Defense gets ball at 80 yards (their 20-yard line)")
    else:
        defense_yardline = 100 - int_catch_location
        print(f"  INT at {int_catch_location} (offense perspective)")
        print(f"  Defense has {defense_yardline} yards to their end zone")
        final_defense_position = defense_yardline - avg_int_return
        print(f"  After avg return ({avg_int_return:.2f} yds): {final_defense_position:.1f} yards")
        if final_defense_position <= 0:
            print(f"  Would be pick-six, use halfway: {defense_yardline / 2:.1f} yards")
        else:
            print(f"  Final position: {final_defense_position:.1f} yards")
    
    # Expected EP calculation
    actual_defenders = row['num_defenders_in_zone']
    defender_reached = row['target_made_zone']
    
    if not defender_reached:
        if actual_defenders == 0:
            no_reach_def_cat = '0'
            reach_def_cat = '1'
            reach_split = 1
        else:
            no_reach_def_cat = str(min(actual_defenders, 1))
            reach_def_cat = '2+' if actual_defenders >= 1 else '1'
            reach_split = 2
    else:
        if actual_defenders == 1:
            no_reach_def_cat = '0'
            reach_def_cat = '1'
            reach_split = 1
        else:
            no_reach_def_cat = '1'
            reach_def_cat = '2+'
            reach_split = actual_defenders
    
    print(f"\nEXPECTED EP CALCULATION:")
    print(f"  Credit Split Factor: {reach_split} (divide by this if reach zone)")
    
    probs_no_reach = outcome_prob_dict[no_reach_def_cat]
    print(f"  If DOESN'T reach zone ({no_reach_def_cat} defenders):")
    print(f"    P(C)={probs_no_reach['C']:.3f}, P(I)={probs_no_reach['I']:.3f}, P(IN)={probs_no_reach['IN']:.3f}")
    ep_no_reach = (
        probs_no_reach['C'] * ep_c +
        probs_no_reach['I'] * ep_i +
        probs_no_reach['IN'] * ep_in
    )
    print(f"    → Expected EP: {ep_no_reach:.3f}")
    
    probs_reach = outcome_prob_dict[reach_def_cat]
    print(f"  If REACHES zone ({reach_def_cat} defender(s)):")
    print(f"    P(C)={probs_reach['C']:.3f}, P(I)={probs_reach['I']:.3f}, P(IN)={probs_reach['IN']:.3f}")
    ep_reach_raw = (
        probs_reach['C'] * ep_c +
        probs_reach['I'] * ep_i +
        probs_reach['IN'] * ep_in
    )
    ep_reach = ep_reach_raw / reach_split
    print(f"    → Expected EP (before split): {ep_reach_raw:.3f}")
    print(f"    → Expected EP (after ÷{reach_split}): {ep_reach:.3f}")
    
    reach_prob = row['pred_prob']
    expected_ep = (1 - reach_prob) * ep_no_reach + reach_prob * ep_reach
    print(f"\n  Weighted Expected EP:")
    print(f"    (1 - {reach_prob:.3f}) × {ep_no_reach:.3f} + {reach_prob:.3f} × {ep_reach:.3f}")
    print(f"    = {expected_ep:.3f}")
    
    # Actual EP calculation
    print(f"\nACTUAL EP CREATED:")
    actual_outcome = row['pass_result']
    
    if defender_reached:
        print(f"  Defender reached zone → Use actual outcome ({actual_outcome})")
        if actual_outcome == 'C':
            actual_ep_raw = ep_c
        elif actual_outcome == 'I':
            actual_ep_raw = ep_i
        else:
            actual_ep_raw = ep_in
        
        if actual_defenders > 1:
            actual_ep = actual_ep_raw / actual_defenders
            print(f"  Actual EP (before split): {actual_ep_raw:.3f}")
            print(f"  Actual EP (after ÷{actual_defenders}): {actual_ep:.3f}")
        else:
            actual_ep = actual_ep_raw
            print(f"  Actual EP: {actual_ep:.3f}")
    else:
        if actual_defenders == 0:
            def_cat = '0'
        elif actual_defenders == 1:
            def_cat = '1'
        else:
            def_cat = '2+'
        
        print(f"  Defender didn't reach → Use probabilities for {def_cat} defenders")
        probs = outcome_prob_dict[def_cat]
        actual_ep = (
            probs['C'] * ep_c +
            probs['I'] * ep_i +
            probs['IN'] * ep_in
        )
        print(f"  Actual EP: {actual_ep:.3f}")
    
    # Final DRPVOE
    drpvoe = actual_ep - expected_ep
    print(f"\n DRPVOE (Defensive Reactionary Playmaking Value Over Expected):")
    print(f"  {actual_ep:.3f} - {expected_ep:.3f} = {drpvoe:+.3f}")
    
    if drpvoe > 0:
        print(f"  ✓ Defender created {drpvoe:.3f} more EP than expected!")
    else:
        print(f"  ✗ Defender created {abs(drpvoe):.3f} fewer EP than expected")
    
    print("="*100)
    print("\n\n")

# Selecting 4 different types of plays to audit
# 1. Catch with 4th down turnover
fourth_down_catch_no_convert = df_metric[
    (df_metric['down'] == 4) & 
    (df_metric['pass_result'] == 'C')
]
if len(fourth_down_catch_no_convert) > 0:
    for idx, play in fourth_down_catch_no_convert.iterrows():
        catch_location = play['yardline'] - play['pass_length']
        expected_yac = yac_dict.get(play['def_category'], 0)
        final_position = catch_location - expected_yac
        if final_position <= 0:
            catch_yardline = catch_location / 2
        else:
            catch_yardline = final_position
        catch_yardline = max(catch_yardline, 1)
        yardline_to_gain = play['yardline'] - play['yards_to_go']
        
        if catch_yardline > yardline_to_gain:  # Didn't convert
            print("CATCH ON 4TH DOWN - TURNOVER ON DOWNS:")
            audit_play_calculation_detailed(play, ep_model, yac_dict, avg_int_return_yards, outcome_prob_dict)
            break
else:
    print("No 4th down catch plays found\n\n")

# 2. Catch without first down
catch_no_first_down = df_metric[
    (df_metric['down'] < 4) & 
    (df_metric['pass_result'] == 'C')
]
if len(catch_no_first_down) > 0:
    for idx, play in catch_no_first_down.iterrows():
        catch_location = play['yardline'] - play['pass_length']
        expected_yac = yac_dict.get(play['def_category'], 0)
        final_position = catch_location - expected_yac
        if final_position <= 0:
            catch_yardline = catch_location / 2
        else:
            catch_yardline = final_position
        catch_yardline = max(catch_yardline, 1)
        yardline_to_gain = play['yardline'] - play['yards_to_go']
        
        if catch_yardline > yardline_to_gain:
            print("CATCH WITHOUT FIRST DOWN CONVERSION:")
            audit_play_calculation_detailed(play, ep_model, yac_dict, avg_int_return_yards, outcome_prob_dict)
            break
else:
    print("No catch without first down found\n\n")

# 3. Interception where defender reached zone
int_reach_play = df_metric[(df_metric['target_made_zone'] == 1) & 
                            (df_metric['pass_result'] == 'IN')]
if len(int_reach_play) > 0:
    print("INTERCEPTION (Defender Reached Zone):")
    audit_play_calculation_detailed(int_reach_play.iloc[0], ep_model, yac_dict, avg_int_return_yards, outcome_prob_dict)

# 4. Play with multiple defenders in zone
multi_def_play = df_metric[df_metric['num_defenders_in_zone'] >= 2]
if len(multi_def_play) > 0:
    print("PLAY WITH MULTIPLE DEFENDERS IN ZONE:")
    audit_play_calculation_detailed(multi_def_play.iloc[0], ep_model, yac_dict, avg_int_return_yards, outcome_prob_dict)

CATCH ON 4TH DOWN - TURNOVER ON DOWNS:
PLAY AUDIT: Divine Deablo - Game 2023091701, Play 2465

INITIAL PLAY STATE:
  Down: 4
  Yards to Go: 2
  Yardline (yards to end zone): 2
  Yardline to Gain (first down marker): 0
  Pass Length (air yards): 2
  Score Diff: 11
  Time Remaining Game: 2166 seconds
  Time Remaining Half: 1266 seconds
  Quarter: 3
  End of Half Flag: 0
  Hurry Up Time Flag: 0
  Defender Category (# def in zone): 1
  Defender Reached Zone: False
  Total Defenders in Zone: 1
  Actual Outcome: C
  Probability of Reaching Zone: 0.167

EXPECTED POINTS BY OUTCOME:
  Completion (C):    -7.000 EP
  Incompletion (I):  -0.511 EP
  Interception (IN):  0.366 EP

CATCH OUTCOME DETAILS:
  Catch Location: 2 - 2 = 0 yards to end zone
  Ball caught in end zone = TOUCHDOWN (-7.0 EP)

INCOMPLETION OUTCOME DETAILS:
  4TH DOWN INCOMPLETION = TURNOVER ON DOWNS
   Defense gets ball at 98 yards (their perspective)

INTERCEPTION OUTCOME DETAILS:
  INT in end zone = TOUCHBACK
  Defense gets ball

In [82]:
import pandas as pd

# Create leaderboard dataframe
leaderboard = player_totals.reset_index()
leaderboard['rank'] = range(1, len(leaderboard) + 1)
leaderboard['drpvoe_per_play'] = leaderboard['drpvoe'] / leaderboard['n_plays']

def format_leaderboard(df, title):
    print("=" * 100)
    print(f"{title:^100}")
    print("=" * 100)
    print(f"{'Rank':<6} {'Player Name':<30} {'NFL ID':<12} {'drpvoe':>10} {'Plays':>8} {'DRPVOE/Play':>12}")
    print("-" * 100)
    
    for idx, row in df.iterrows():
        rank = row['rank']
        name = row['player_name'][:29]  # Truncate long names
        nfl_id = row['nfl_id']
        drpvoe = row['drpvoe']
        plays = int(row['n_plays'])
        drpvoe_per = row['drpvoe_per_play']
        
        print(f"{rank:<4} {name:<30} {nfl_id:<12} {drpvoe:>+9.2f} {plays:>8,} {drpvoe_per:>+11.4f}")
    
    print("=" * 100)
    print()

# Top 20 Players
print("\n\n")
top_20 = leaderboard.head(20)
format_leaderboard(top_20, "TOP 20 DEFENDERS - DEFENSIVE REACTIONARY PLAYMAKING VALUE OVER EXPECTED (DRPVOE)")

# Bottom 20 Players
bottom_20 = leaderboard.tail(20).sort_values('rank')
format_leaderboard(bottom_20, "BOTTOM 20 DEFENDERS - DEFENSIVE REACTIONARY PLAYMAKING VALUE OVER EXPECTED (DRPVOE)")

# Summary Statistics
print("=" * 100)
print(f"{'METRIC SUMMARY STATISTICS':^100}")
print("=" * 100)
print(f"\n{'SAMPLE SIZE':<30}")
print(f"  Total Players:              {len(leaderboard):>8,}")
print(f"  Total Plays Analyzed:       {leaderboard['n_plays'].sum():>8,}")
print(f"  Avg Plays per Player:       {leaderboard['n_plays'].mean():>8.1f}")
print(f"  Median Plays per Player:    {leaderboard['n_plays'].median():>8.0f}")

print(f"\n{'DRPVOE DISTRIBUTION':<30}")
print(f"  Mean:                       {leaderboard['drpvoe'].mean():>+8.3f}")
print(f"  Median:                     {leaderboard['drpvoe'].median():>+8.3f}")
print(f"  Std Dev:                    {leaderboard['drpvoe'].std():>8.3f}")
print(f"  25th Percentile:            {leaderboard['drpvoe'].quantile(0.25):>+8.3f}")
print(f"  75th Percentile:            {leaderboard['drpvoe'].quantile(0.75):>+8.3f}")

print(f"\n{'EXTREMES':<30}")
best_player = leaderboard.loc[leaderboard['drpvoe'].idxmax()]
worst_player = leaderboard.loc[leaderboard['drpvoe'].idxmin()]
print(f"  Best:                       {best_player['drpvoe']:>+8.3f} ({best_player['player_name']})")
print(f"  Worst:                      {worst_player['drpvoe']:>+8.3f} ({worst_player['player_name']})")

print(f"\n{'DRPVOE PER PLAY DISTRIBUTION':<30}")
print(f"  Mean:                       {leaderboard['drpvoe_per_play'].mean():>+8.4f}")
print(f"  Median:                     {leaderboard['drpvoe_per_play'].median():>+8.4f}")
print(f"  Std Dev:                    {leaderboard['drpvoe_per_play'].std():>8.4f}")

print(f"\n{'PERFORMANCE BREAKDOWN':<30}")
positive_drpvoe = (leaderboard['drpvoe'] > 0).sum()
negative_drpvoe = (leaderboard['drpvoe'] < 0).sum()
neutral_drpvoe = (leaderboard['drpvoe'] == 0).sum()
print(f"  Positive DRPVOE:             {positive_drpvoe:>8,} ({100*positive_drpvoe/len(leaderboard):.1f}%)")
print(f"  Negative DRPVOE:             {negative_drpvoe:>8,} ({100*negative_drpvoe/len(leaderboard):.1f}%)")
print(f"  Neutral DRPVOE:              {neutral_drpvoe:>8,} ({100*neutral_drpvoe/len(leaderboard):.1f}%)")

print("=" * 100)
print()

# Create visualizations
print("\n{'DISTRIBUTION SNAPSHOT':<30}")
bins = [-float('inf'), -10, -5, -2, -1, 0, 1, 2, 5, 10, float('inf')]
labels = ['<-10', '-10 to -5', '-5 to -2', '-2 to -1', '-1 to 0', '0 to 1', '1 to 2', '2 to 5', '5 to 10', '>10']
leaderboard['drpvoe_bin'] = pd.cut(leaderboard['drpvoe'], bins=bins, labels=labels)
distribution = leaderboard['drpvoe_bin'].value_counts().sort_index()

print(f"\n  {'Range':<12} {'Count':<8} {'Pct':<8} {'Bar'}")
print(f"  {'-'*50}")
for bin_label, count in distribution.items():
    pct = 100 * count / len(leaderboard)
    bar = '█' * int(pct / 2)  # Scale to fit
    print(f"  {bin_label:<12} {count:<8} {pct:>5.1f}%   {bar}")

# Save to CSV
leaderboard_export = leaderboard[['rank', 'player_name', 'nfl_id', 'drpvoe', 'n_plays', 'drpvoe_per_play']]
leaderboard_export.columns = ['Rank', 'Player Name', 'NFL ID', 'Total DRPVOE', 'Plays', 'DRPVOE per Play']
leaderboard_export.to_csv('drpvoe_leaderboard_final.csv', index=False)




          TOP 20 DEFENDERS - DEFENSIVE REACTIONARY PLAYMAKING VALUE OVER EXPECTED (DRPVOE)          
Rank   Player Name                    NFL ID           drpvoe    Plays  DRPVOE/Play
----------------------------------------------------------------------------------------------------
1    Bobby Okereke                  47872           +19.81       71     +0.2790
2    Rasul Douglas                  44911           +17.24       91     +0.1894
3    Julian Blackmon                52493           +16.76      112     +0.1496
4    Jaylon Johnson                 52458           +15.90       63     +0.2524
5    Charvarius Ward                46757           +14.77       98     +0.1507
6    Cam Taylor-Britt               54525           +14.55       66     +0.2204
7    Paulson Adebo                  53505           +14.17       89     +0.1592
8    Kerby Joseph                   54562           +14.16       76     +0.1863
9    Geno Stone                     52627           +13.66       82    

In [86]:
# Create play-level leaderboard
play_leaderboard = metric_df.copy()

# Merge with df_metric to get play context
play_details = df_metric[['game_id', 'play_id', 'nfl_id', 'player_name', 'down', 'yards_to_go', 
                           'yardline', 'pass_result', 'target_made_zone', 'num_defenders_in_zone', 
                           'pred_prob', 'pass_length']].drop_duplicates()

play_leaderboard = play_leaderboard.merge(play_details, on=['game_id', 'play_id', 'nfl_id', 'player_name'], how='left')

def format_play_leaderboard(df, title):
    """Format a nice looking play leaderboard"""
    
    print("=" * 150)
    print(f"{title:^150}")
    print("=" * 150)
    print(f"{'Player':<25} {'Game ID':<12} {'Play ID':<10} {'Down':<8} {'Dist':<6} {'Yd Ln':<7} {'Pass':<6} {'Outcome':<8} {'Reached':<9} {'#Def':<6} {'P(Reach)':<10} {'drpvoe':>10}")
    print("-" * 150)
    
    for idx, row in df.iterrows():
        name = row['player_name'][:24]
        game_id = str(row['game_id'])
        play_id = str(row['play_id'])
        down_dist = f"{int(row['down'])} & {int(row['yards_to_go'])}"
        yardline = f"{int(row['yardline'])}"
        pass_len = f"{int(row['pass_length']) if pd.notna(row['pass_length']) else 0}"
        outcome = row['pass_result']
        reached = "Yes" if row['target_made_zone'] else "No"
        num_def = int(row['num_defenders_in_zone'])
        prob = row['pred_prob']
        drpvoe = row['drpvoe']
        
        print(f"{name:<23} {game_id:<12} {play_id:<10} {down_dist:<8} {int(row['yards_to_go']):<6} {yardline:<7} {pass_len:<6} {outcome:<8} {reached:<9} {num_def:<6} {prob:<9.3f} {drpvoe:>+9.3f}")
    
    print("=" * 150)
    print()

# Top 10 plays
print("\n\n")
top_10_plays = play_leaderboard.nlargest(10, 'drpvoe')
format_play_leaderboard(top_10_plays, "TOP 10 INDIVIDUAL PLAYS - HIGHEST DRPVOE")

# Bottom 10 plays
bottom_10_plays = play_leaderboard.nsmallest(10, 'drpvoe')
format_play_leaderboard(bottom_10_plays, "BOTTOM 10 INDIVIDUAL PLAYS - LOWEST DRPVOE")

# Save to CSV
play_leaderboard_export = play_leaderboard.sort_values('drpvoe', ascending=False).copy()
play_leaderboard_export['rank'] = range(1, len(play_leaderboard_export) + 1)
play_export_cols = ['rank', 'player_name', 'game_id', 'play_id', 'down', 'yards_to_go', 'yardline', 
                    'pass_length', 'pass_result', 'target_made_zone', 'num_defenders_in_zone', 
                    'pred_prob', 'actual_ep', 'expected_ep', 'drpvoe']
play_leaderboard_export[play_export_cols].to_csv('drpvoe_play_leaderboard.csv', index=False)




                                                       TOP 10 INDIVIDUAL PLAYS - HIGHEST DRPVOE                                                       
Player                    Game ID      Play ID    Down     Dist   Yd Ln   Pass   Outcome  Reached   #Def   P(Reach)       drpvoe
------------------------------------------------------------------------------------------------------------------------------------------------------
Bobby Okereke           2023100804   1755       3 & 4    4      4       4      IN       Yes       1      0.130        +7.074
Elijah Molden           2023121703   1053       3 & 10   10     60      4      IN       Yes       1      0.287        +6.361
Trenton Thompson        2023112601   2200       3 & 7    7      18      3      IN       Yes       1      0.167        +6.343
Tre'Davious White       2023092408   2268       2 & 8    8      15      15     IN       Yes       1      0.042        +6.253
Sam Franklin Jr.        2023100102   313        3 & 5    5      5 